Mise a jour du système:

In [ ]:
#! /bin/bash
sudo apt update && sudo apt upgrade -y

Installation de paquet necessaire

In [ ]:
#! /bin/bash
sudo apt install curl git gh vim nano jq

Mise en place de l'utilisateur

In [ ]:
#!/bin/bash
sudo adduser db_user  --system --comment 'MariaDB App system-user' --home /home/db_user --shell /bin/bash

Mod de passe de l'utilisateur : 

In [ ]:
#!/bin/bash
sudo passwd db_user
# esiroi974

## Partie 4 : Déploiment de l'API

### MariaDB

Installation de la DB

In [ ]:
#! /bin/bash
sudo apt install software-properties-common mariadb-server

Démarrage avec le serveur

In [ ]:
#! /bin/bash
sudo systemctl enable mariadb

Démarrer maintenant

In [ ]:
#!/bin/bash
sudo systemctl start mariadb

Accès a MariaDB et création des tables:

In [ ]:
#!/bin/bash
sudo mariadb

In [ ]:
CREATE DATABASE pollution_db;
GRANT ALL PRIVILEGES ON pollution_db.* TO 'db_user'@'localhost' IDENTIFIED BY 'esiroi974';
GRANT ALL PRIVILEGES ON pollution_db.* TO 'db_user'@'%'         IDENTIFIED BY 'esiroi974';
FLUSH PRIVILEGES;

In [ ]:
CREATE TABLE pollution_data (
    object_id INT PRIMARY KEY,            -- Identifiant unique de chaque ligne, clé primaire
    station_id VARCHAR(255),              -- Identifiant de la station
    pollutant VARCHAR(255),               -- Nom du polluant
    concentration FLOAT,                  -- Concentration du polluant
    timestamp DATETIME,                   -- Date et heure de la mesure
    typologie VARCHAR(255),               -- Typologie de la station (ex. urbain, industriel)
    nom_station VARCHAR(255),             -- Nom de la station
    unite VARCHAR(50),                    -- Unité de mesure (ex. µg/m³)
    x FLOAT,                        -- Coordonnées géographiques (en x)
    y FLOAT                         -- Coordonnées géographiques (en y)
);

SHOW COLUMNS IN pollution_data;

Accès a la BDD

In [ ]:
#!/bin/bash
mariadb -h localhost -u db_user -pesiroi974 -D pollution_db

Installation de Python et des dépendances mariaDB

In [ ]:
#!/bin/bash
sudo apt install python3 python3-pip python3-venv

In [ ]:
#!/bin/bash
sudo apt install mariadb-client libmariadb-dev pkg-config

### Installation de l'appli

GIT clone

In [ ]:
#!/bin/bash
# Connection a vapormap
~/vapormap-prod
git clone https://github.com/gitreunion/datacup-challenge-esi-dev.git

Environnement python : 

In [ ]:
#!/bin/bash
cd ~/datacup-challenge-esi-dev
mkdir venv
python3 -m venv --prompt vapormap-prod ./venv
source venv/bin/activate

Installation des pré-requis

In [ ]:
#!/bin/bash
pip install wheel
cd ~/vapormap-prod/api/
pip install -r requirements/production.txt

Mise en place des variable d'environnement : 

In [ ]:
#!/bin/bash
export VAPOR_DBNAME='db_vapormap'
export VAPOR_DBUSER='user_vapormap'
export VAPOR_DBPASS='vapormap'
export VAPOR_DBHOST='localhost'
export FLASK_APP="app"
export SETTINGS_FILE="production"

Initialisation de la DB

In [ ]:
#!/bin/bash
cd ~/vapormap-prod/api/app
flask db upgrade

In [ ]:
#! /bin/bash
export PYTHONDONTWRITEBYTECODE=1
export PYTHONUNBUFFERED=1
export PYTHONPATH=$HOME/vapormap-prod/api

In [ ]:
#! /bin/bash
cd ~/vapormap-prod/api
gunicorn --bind 0.0.0.0:5000 wsgi:app
bg 1
curl http://localhost:5000/api/points/?format=json
#Renvoie []
fg 1
#CTRL C


## Partie 5

In [ ]:
#! /bin/bash
cd /home/app-vapormap/vapormap-prod/api

export VAPORMAP_PATH='/home/app-vapormap/vapormap-prod/venv/bin'
export VAPORMAP_DIR='/home/app-vapormap/vapormap-prod/api'
export VAPORMAP_USER='app-vapormap'
export VAPORMAP_GROUP='nogroup'
export VAPOR_DBNAME='db_vapormap'
export VAPOR_DBUSER='user_vapormap'
export VAPOR_DBPASS='vapormap'
export VAPOR_DBHOST='localhost'
export VAPORMAP_API_PORT=5000
export FLASK_APP="app"
export SETTINGS_FILE="production"
export PYTHONDONTWRITEBYTECODE=1
export PYTHONUNBUFFERED=1
export PYTHONPATH=$HOME/vapormap-prod/api


La partie suivante ce fait en root

In [ ]:
#! /bin/bash
# Creation du fichier
cp api-systemd.conf.template /etc/systemd/system/api-systemd.conf.template
cd /etc/systemd/system
envsubst < api-systemd.conf.template > api-systemd.service
chmod 755 api-systemd.service
# Activation du service
sudo systemctl daemon-reload
sudo systemctl enable api-systemd.service
sudo systemctl start api-systemd.service
sudo systemctl status api-systemd.service

Test : 

In [ ]:
#! /bin/bash
curl http://localhost:5000/api/points/?format=json
# Renvoie []

Sortie du root

## Partie 6:

In [ ]:
#! /bin/bash
sudo apt install nginx-light
sudo systemctl enable nginx

Sur l'utilisateur vapor map : 

In [ ]:
#! /bin/bash
cd ~/vapormap-prod/frontend
export VAPORMAP_BACKEND_HOST=localhost
export VAPORMAP_BACKEND_PORT=5000
envsubst < config.json.template > html/config.json
cat html/config.json


Pour NGINX : 

- Passage sur root

In [ ]:
#! /bin/bash
export VAPORMAP_URL_SERVERNAME=0.0.0.0
export VAPORMAP_URL_PORT=8000
export VAPORMAP_FRONTEND_ROOT=/home/app-vapormap/vapormap-prod/frontend/html
export uri='$uri' #Evite un méchant bug remplacant $uri par du vide lors du envsubst (1h pour comprendre le bug , j'apprecie mon existence)
envsubst < /home/app-vapormap/vapormap-prod/frontend/nginx.conf.template > /etc/nginx/sites-available/vapormap
cat /etc/nginx/sites-available/vapormap 

Creation du lien symbolique

In [ ]:
#! /bin/bash
ln -s /etc/nginx/sites-available/vapormap /etc/nginx/sites-enabled/vapormap

Vérification et relance de Nginx

In [ ]:
#! /bin/bash
nginx -t
systemctl restart nginx

Modification de la configuration de Nginx: 

In [ ]:
#! /bin/bash
# Pour savoir sous quel utilisateur/groupe tourne le service (merci internet)
ps aux | grep nginx


# Déterminer l'utilisateur et le groupe sous lesquels s'exécute Nginx

sudo chmod 755 /home/app-vapormap
sudo chmod 755 /home/app-vapormap/vapormap-prod

# Autoriser l'accès en lecture au dossier
sudo chgrp -R www-data /home/app-vapormap/vapormap-prod/frontend
sudo chmod -R 750 /home/app-vapormap/vapormap-prod/frontend

# Redémarrer Nginx
sudo systemctl restart nginx

## Partie 7 : Vérification

In [ ]:
#! /bin/bash
# Vérifier que le site est accessible
curl http://localhost:8000
curl http://localhost:5000/api/points/?format=json

Il faut ensuite vérifier le bon fonctionnement du site : 

![](./image/1.png)

In [ ]:
#! /bin/bash  
# On reboot

sudo reboot

Après reboot le site affiche toujours les memes points , et fonctionne sans devoir redemarrer les services manuellement ! 

Vérification de la DB : 

In [ ]:
#! /bin/bash
mariadb -h localhost -u user_vapormap -pvapormap -D db_vapormap

In [ ]:
SHOW TABLES;
SELECT * FROM point; 
-- La table api_point est ici plutot la table point
-- Sortie : 
-- +----------+--------------+-----------+----------+---------+---------------------+
-- | point_id | element_name | longitude | latitude | comment | date                |
-- +----------+--------------+-----------+----------+---------+---------------------+
-- |        1 | Hey          |   57.3926 |  35.4607 | NULL    | 2024-11-06 00:00:00 |
-- |        2 | Hey          |   44.1211 |  20.6328 | NULL    | 2024-11-06 00:00:00 |
-- |        3 | Hey          |    9.4043 |  46.6796 | NULL    | 2024-11-06 00:00:00 |
-- |        4 | Hey          |   35.2441 |  51.6726 | NULL    | 2024-11-06 00:00:00 |
-- |        5 | Hey          |    49.043 |  47.8721 | NULL    | 2024-11-06 00:00:00 |
-- |        7 | Heydsd       |   15.4688 | -13.9234 | NULL    | 2024-11-06 00:00:00 |
-- |        8 | dffd         |   78.5742 |  22.2688 | NULL    | 2024-11-21 00:00:00 |
-- +----------+--------------+-----------+----------+---------+---------------------+

## Partie 8:

In [ ]:
#! /bin/bash
cd /home/app-vapormap/vapormap-prod/api
# Modifions plutot le template avec nano : 
nano api-systemd.conf.template 
#Créons les fichier par nous même (gunicorn n'a pas l'air des les créer)
mkdir /var/log/vapormap/ && touch /var/log/vapormap/gunicorn-access.log
touch /var/log/vapormap/gunicorn-error.log
chown app-vapormap /var/log/vapormap/gunicorn-access.log
chown app-vapormap /var/log/vapormap/gunicorn-error.log
# On modifie l'execution de cette manière :
${VAPORMAP_PATH}/gunicorn wsgi:app --bind 0.0.0.0:${VAPORMAP_API_PORT} --access-logfile /var/log/vapormap/gunicorn-access.log --error-logfile /var/log/vapormap/gunicorn-error.log
# Sauvegarder et redemarrer le service : 
#(En sudo)
cp /home/app-vapormap/vapormap-prod/api/api-systemd.conf.template /etc/systemd/system/api-systemd.conf.template
cd /etc/systemd/system
envsubst < api-systemd.conf.template > api-systemd.service
chmod 755 api-systemd.service
# Activation du service
sudo systemctl daemon-reload
sudo systemctl restart api-systemd.service
sudo systemctl status api-systemd.service

En créant volontairement des logs et erreur, les fichiers de logs sont bien rempli : 

In [ ]:
cat /var/log/vapormap/gunicorn-access.log
# 10.0.2.2 - - [13/Nov/2024:19:34:36 +0400] "POST /api/points/ HTTP/1.1" 200 132 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:36 +0400] "GET /geojson HTTP/1.1" 200 835 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:40 +0400] "POST /api/points/ HTTP/1.1" 200 138 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:40 +0400] "GET /geojson HTTP/1.1" 200 1001 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:43 +0400] "POST /api/points/ HTTP/1.1" 500 31 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:43 +0400] "POST /api/points/ HTTP/1.1" 500 31 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"
# 10.0.2.2 - - [13/Nov/2024:19:34:43 +0400] "POST /api/points/ HTTP/1.1" 500 31 "-" "Mozilla/5.0 (X11; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0"

cat /var/log/vapormap/gunicorn-error.log 

# [2024-11-13 19:33:18 +0400] [3881] [INFO] Starting gunicorn 23.0.0
# [2024-11-13 19:33:18 +0400] [3881] [INFO] Listening at: http://0.0.0.0:5000 (3881)
# [2024-11-13 19:33:18 +0400] [3881] [INFO] Using worker: sync
# [2024-11-13 19:33:18 +0400] [3882] [INFO] Booting worker with pid: 3882
# [2024-11-13 19:34:36,246] WARNING in app: POST
# [2024-11-13 19:34:40,840] WARNING in app: POST
# [2024-11-13 19:34:43,096] WARNING in app: POST
# [2024-11-13 19:34:43,097] WARNING in app: (MySQLdb.DataError) (1265, "Data truncated for column 'longitude' at row 1")
# [SQL: INSERT INTO point (element_name, longitude, latitude, comment, date) VALUES (%s, %s, %s, %s, %s)]
# [parameters: ('dffdqdsfsdfdsds', '60.4687cdvd5000000001', '-4.915832801313164', None, datetime.datetime(2024, 11, 21, 0, 0))]
# (Background on this error at: https://sqlalche.me/e/20/9h9h)
# [2024-11-13 19:34:43,636] WARNING in app: POST
# [2024-11-13 19:34:43,638] WARNING in app: (MySQLdb.DataError) (1265, "Data truncated for column 'longitude' at row 1")
# [SQL: INSERT INTO point (element_name, longitude, latitude, comment, date) VALUES (%s, %s, %s, %s, %s)]
# [parameters: ('dffdqdsfsdfdsds', '60.4687cdvd5000000001', '-4.915832801313164', None, datetime.datetime(2024, 11, 21, 0, 0))]
# (Background on this error at: https://sqlalche.me/e/20/9h9h)
# [2024-11-13 19:34:43,811] WARNING in app: POST
# [2024-11-13 19:34:43,812] WARNING in app: (MySQLdb.DataError) (1265, "Data truncated for column 'longitude' at row 1")
# [SQL: INSERT INTO point (element_name, longitude, latitude, comment, date) VALUES (%s, %s, %s, %s, %s)]
# [parameters: ('dffdqdsfsdfdsds', '60.4687cdvd5000000001', '-4.915832801313164', None, datetime.datetime(2024, 11, 21, 0, 0))]
# (Background on this error at: https://sqlalche.me/e/20/9h9h)



## Partie 9:

Modifions de nouveau le fichier template : 

In [ ]:
#! /bin/bash
nano /home/app-vapormap/vapormap-prod/frontend/nginx.conf.template
# On modifie en ajoutant:

# Backend API
        location /api/ {
            proxy_set_header Host ;
            proxy_set_header X-Real-IP ;
            proxy_set_header X-Scheme ;
            proxy_set_header X-Forwarded-Proto ;
            proxy_set_header X-Forwarded-For ;
            proxy_pass http://127.0.0.1:5000/api/;
            proxy_redirect off;
        }

        # Backend geojson
        location /geojson {
            proxy_set_header Host ;
            proxy_set_header X-Real-IP ;
            proxy_set_header X-Scheme ;
            proxy_set_header X-Forwarded-Proto ;
            proxy_set_header X-Forwarded-For ;
            proxy_pass http://127.0.0.1:5000/geojson;
            proxy_redirect off;
        }
# On prend en compte et on redemarre le service : 

export VAPORMAP_URL_SERVERNAME=0.0.0.0
export VAPORMAP_URL_PORT=8000
export VAPORMAP_FRONTEND_ROOT=/home/app-vapormap/vapormap-prod/frontend/html
# Manière encore plus détourné pour éviter des problèmes avec envsubst (merci StackOverflow)
envsubst '$VAPORMAP_URL_SERVERNAME $VAPORMAP_URL_PORT $VAPORMAP_FRONTEND_ROOT' < /home/app-vapormap/vapormap-prod/frontend/nginx.conf.template > /etc/nginx/sites-available/vapormap
cat /etc/nginx/sites-available/vapormap 
nginx -t
systemctl restart nginx


Pour config.js repassons sur l'utilisateur vapormap:

In [ ]:
#! /bin/bash
cd ~/vapormap-prod/frontend
export VAPORMAP_BACKEND_HOST=localhost
export VAPORMAP_BACKEND_PORT=8000
envsubst < config.json.template > html/config.json
cat html/config.json
curl http://localhost:8000/api/points/?format=json
# Renvoie du json

Pas besoin de changer la configuration de gunicorn il n'écoute que sur localhost.

In [ ]:
#! /bin/bash
#Verification : 
sudo tail -f /var/log/nginx/access.log
sudo tail -f /var/log/nginx/error.log

cat /var/log/vapormap/gunicorn-access.log
cat /var/log/vapormap/gunicorn-error.log 

